Pre-processing of data including: data screening, identification of outliers, weighted average of data based on sample size, gridding  

对数据进行预处理，包括：数据筛选，识别异常值，根据样本数对数据进行加权平均，网格化  

In [ ]:
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
import myfunction as mf
path_data_raw = "C:/Users/dell/OneDrive/file/"
path_country_nc = "C:/Users/dell/OneDrive/file/nc/"
path_one_spdb = 'C:/Users/dell/OneDrive/file/SPDB/'

drive_letter = 'E:'
path_opp = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/pretreatment/"
path_pre = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/pretreatment/"
path_match = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/match/"
path_picture = drive_letter + "/wyy/code_project/running_outcome/final_fig/SPDB/part0_treat/pretreatment/"
path_temp = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/temp/"

path_ng_fig = drive_letter + "/wyy/code_project/running_outcome/final_fig/ng_fig/"
list_color = ["#ee877c", "#8bd0e3", "#6abeae", "#808eaf", "#f7bba8", "#acb4cc", "#b5e0d5", "#e86462", "#a89687"]

In [ ]:
# Preparation of functions to identify outliers
# 识别异常值的函数准备

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from pyod.models.iforest import IForest
import matplotlib.pyplot as plt

def get_outlier(df_x, col_1, col_2, col_3, int_fraction):
    """
    col_1, col_2, col_3: Three variables for identifying outliers
    int_fraction: Percentage of outliers
    ---
    col_1, col_2, col_3: 帮助识别异常值的三个变量
    int_fraction: 异常值百分比
    """
    df = df_x.copy()
    scaler = MinMaxScaler(feature_range=(0, 1))
    df[[col_1, col_2, col_3]] = scaler.fit_transform(df[[col_1, col_2, col_3]])
    df[[col_1, col_2, col_3]].head()
    X1 = df[col_1].values.reshape(-1, 1)
    X2 = df[col_2].values.reshape(-1, 1)
    X3 = df[col_3].values.reshape(-1, 1)
    X = np.concatenate((X1, X2, X3), axis=1)
    random_state = np.random.RandomState(1024)
    outliers_fraction = int_fraction
    classifiers = {
        'Isolation Forest': IForest(contamination=outliers_fraction, random_state=random_state),
    }
    xx, yy, zz = np.meshgrid(np.linspace(0, 1, 200), np.linspace(0, 1, 200), np.linspace(0, 1, 200))
    for i, (clf_name, clf) in enumerate(classifiers.items()):
        clf.fit(X)
        # 用该模型训练X
        # predict raw anomaly score
        scores_pred = clf.decision_function(X) * -1
        # 计算样本点到分割超平面的函数距离，即各个数据的异常得分
        # prediction of a datapoint category outlier or inlier
        y_pred = clf.predict(X)
        n_inliers = len(y_pred) - np.count_nonzero(y_pred)
        n_outliers = np.count_nonzero(y_pred == 1)
        plt.figure(figsize=(10, 10))
        dfx = df
        dfx['outlier'] = y_pred.tolist()
        IX1 = np.array(dfx[col_1][dfx['outlier'] == 0]).reshape(-1, 1)
        IX2 = np.array(dfx[col_2][dfx['outlier'] == 0]).reshape(-1, 1)
        IX3 = np.array(dfx[col_3][dfx['outlier'] == 0]).reshape(-1, 1)
        OX1 = dfx[col_1][dfx['outlier'] == 1].values.reshape(-1, 1)
        OX2 = dfx[col_2][dfx['outlier'] == 1].values.reshape(-1, 1)
        OX3 = dfx[col_3][dfx['outlier'] == 1].values.reshape(-1, 1)
        print('OUTLIERS : ', n_outliers, 'INLIERS : ', n_inliers, clf_name)
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(IX1, IX2, IX3, c='white', s=6, edgecolor='k')
        ax.scatter(OX1, OX2, OX3, c='red', s=6, edgecolor='k')
        plt.xlim((0, 1))
        plt.ylim((0, 1))
        plt.title(clf_name)
        plt.show()
        return dfx

In [ ]:
# data screening
# 筛选数据

from netCDF4 import Dataset
from scipy.spatial import cKDTree
path_data_raw = "C:/Users/dell/OneDrive/file/"
df_data_raw = pd.read_csv(path_data_raw + "lr_use.csv")
nc_file_path = "C:/Users/dell/OneDrive/file/nc/country_id_0.1x0.1.nc"
nc_data = Dataset(nc_file_path, 'r')
lon = nc_data.variables['lon'][:]
lat = nc_data.variables['lat'][:]
country_id = nc_data.variables['country_id'][:]
lon_lat_grid = np.array(np.meshgrid(lon, lat))
grid_shape = lon_lat_grid.shape[1:]
lon_lat_grid = lon_lat_grid.reshape(2, -1).T
tree = cKDTree(lon_lat_grid)
points_to_query = df_data_raw[['lon', 'lat']].values
distances, indices = tree.query(points_to_query)
df_data_raw['country_id'] = country_id.reshape(-1)[indices]
print(df_data_raw.head())
print(df_data_raw.isnull().sum())
dict_inf = {"sp":['phylum',"class","length_last","weight_last","troph_last", 'rank','family','genus','species'],
             "po":["po_chain","posname",'posid','po_carbon','po_f_carbon','po_classification'],
             'country':['country_code', 'country','continent','income'],
             'habit':['habitat']}
df_data_raw2 = mf.append_inf(df_data_raw, dict_inf)
df_data_raw_select = df_data_raw2[(df_data_raw2['posid'].notna())&(df_data_raw2['rank']=='SPECIES')]
print(df_data_raw2.shape)
print(df_data_raw_select.shape)
print(df_data_raw_select.isnull().sum())
df_data_raw_select = mf.df_merge_lat_lon(df_data_raw_select)
print(df_data_raw_select.dtypes)
df_data_raw_select.to_csv(path_pre + "lr_use_pre_select.csv", index=False)

In [ ]:
# Viewing the data situation
# 查看数据情况
df_data_raw_select = pd.read_csv(path_pre + "lr_use_pre_select.csv")
print(df_data_raw_select.shape)
list_items = ['posname','class','po_chain','habitat','income','continent','organ']
pd.set_option("display.max_rows", None)
for item in list_items:
    # print('-------------',item,'--------------------')
    df_item = mf.col_value(df_data_raw_select,item)
    print(df_item)
    # print('-------------',item,'--------------------')

In [ ]:
# Further screening of data
# 进一步筛选数据
df_data_pre = pd.read_csv(path_pre + "lr_use_pre_select.csv")
print(df_data_pre.shape)
list_posname = mf.col_value(df_data_pre,'posname')['posname'].unique().tolist()[0:15]
list_class = mf.col_value(df_data_pre,'class')['class'].unique().tolist()[0:3]
list_organ = mf.col_value(df_data_pre,'organ')['organ'].unique().tolist()[0:4]
df_data_pre_select = df_data_pre[(df_data_pre['posname'].isin(list_posname))&(df_data_pre['class'].isin(list_class))&(df_data_pre['organ'].isin(list_organ))]
df_data_pre_select['time_year'] = df_data_pre_select['time_year'].astype(float)
df_data_pre_select['time_year'] = df_data_pre_select['time_year'].astype(int)
df_data_pre_select = df_data_pre_select[(df_data_pre_select['time_year']<=2020)&(df_data_pre_select['time_year']>=2000)]
print(df_data_pre_select.shape)
df_data_pre_select.to_csv(path_pre + "lr_select.csv",index=False)


In [ ]:
# Preset one per cent outliers
# 预设百分之一的异常值
df_data_pre_select = pd.read_csv(path_pre + "lr_select.csv")
print(df_data_pre_select.shape)
df_icicle_w_o_outlier = df_data_pre_select.copy()

le_ind = LabelEncoder()
po_le = le_ind.fit_transform(df_icicle_w_o_outlier["posname"])
df_icicle_w_o_outlier["po_copy"] = po_le
df_icicle_w_o_outlier['value_copy'] = (df_icicle_w_o_outlier['value'] - df_icicle_w_o_outlier.groupby('po_copy')['value'].transform('min')) / (df_icicle_w_o_outlier.groupby('po_copy')['value'].transform('max') - df_icicle_w_o_outlier.groupby('po_copy')['value'].transform('min'))
df_icicle_w_o_outlier["time_year_copy"] = df_icicle_w_o_outlier["time_year"]
df_icicle_w_o_outlier = get_outlier(df_icicle_w_o_outlier,"time_year_copy", "po_copy","value_copy", 0.01)

df_icicle_w_o_outlier = df_icicle_w_o_outlier[df_icicle_w_o_outlier["outlier"]==0]

list_3 = ["habitat","spid","posname","lon","lat","organ","time_year", "value", 'n']
list_4 = ["habitat","spid","lon","lat","organ","time_year",'n']
df_icicle_w_o_outlier = df_icicle_w_o_outlier[list_3]
print(df_icicle_w_o_outlier.shape)
df_icicle_w_o_outlier.to_csv(path_pre + "lr_outlier.csv",index=False)



In [ ]:
# weighted average of data based on sample size
# 根据样本数对数据进行加权平均
df_outlier = pd.read_csv(path_pre + "lr_outlier.csv")
print(df_outlier.shape)
list_1 = ["habitat","spid","posname","lon","lat","organ","time_year"]
list_2 = ["habitat","spid","posname","lon","lat","organ","time_year", "n","value"]
list_3 = ["habitat","spid","posname","lon","lat","organ","time_year", "value"]
df_icicle = df_outlier[list_2]
df_icicle_w = mf.get_weight_count(df_icicle,list_1,'mean') 
df_icicle_w_o = df_icicle_w[list_3]
print(df_icicle_w_o.shape)
df_icicle_w_o.to_csv(path_pre + "lr_weight_avg.csv",index=False)

In [ ]:
# Data Gridding
# 数据网格化
df_weight = pd.read_csv(path_pre + "lr_weight_avg.csv")
print(df_weight.shape)
int_grid = 1
def lat_lon_to_grid(lat, lon, grid_size=1):
    return round(lat / grid_size) * grid_size, round(lon / grid_size) * grid_size


df_weight['lat_grid'], df_weight['lon_grid'] = zip(*df_weight.apply(lambda row: lat_lon_to_grid(row['lat'], row['lon'], int_grid), axis=1))

df_grid = df_weight.groupby(['lat_grid', 'lon_grid', 'habitat', 'spid', 'organ', 'time_year', 'posname'])['value'].mean().reset_index()
print(df_grid.shape)
df_grid.to_csv(path_pre + "lr_grid_avg.csv",index=False)

In [ ]:
# Save the dummy code with its corresponding categorical variable
# 保存虚拟编码与之对应的分类变量

list_need_sort = ['class','order','po_classification','posname','organ','habitat',"family","genus","spid"]
dict_inf = {"sp":["class",'order','family','genus','species'],
             "pon":['po_carbon','po_f_carbon','po_classification']}
df_grid = pd.read_csv(path_pre + 'lr_grid.csv')
df_sort = mf.append_inf(df_grid,dict_inf)
for var in list_need_sort:
    df_var = mf.col_describe(df_sort,var)
    df_var = df_var[[var,"50%"]]
    df_var = df_var.sort_values(by="50%", ascending=True)
    df_var = df_var.reset_index(drop=True)
    df_var['id'] = df_var.index
    df_var.to_csv(path_one_spdb + var + '.csv', index=False)

## Water data

In [ ]:
# Filtering data and identifying outliers
# 筛选数据及识别异常值
str_sw_remark = 's7'
df_sw_all = pd.read_csv(path_data_raw + "sw.csv")
list_select_source = ['paper','norman', 'gmp', 'egle']
df_sw_select = df_sw_all[df_sw_all['source'].isin(list_select_source)]
print(df_sw_select.shape)
df_sw_select.to_csv(path_temp + 'sw'+str_sw_remark+'_select.csv', index=False)

df_data_pre_select = pd.read_csv(path_temp + 'sw'+str_sw_remark+'_select.csv')

df_data_pre_select = df_data_pre_select.rename(columns={'sw_value': 'value'})
df_data_pre_select = df_data_pre_select[df_data_pre_select['value']>0]
print(df_data_pre_select.shape)
df_icicle_w_o_outlier = df_data_pre_select.copy()
print(df_icicle_w_o_outlier.dtypes)

df_icicle_w_o_outlier["po_copy"] = df_icicle_w_o_outlier["posname"]
df_icicle_w_o_outlier['value_copy'] = (df_icicle_w_o_outlier['value'] - df_icicle_w_o_outlier.groupby('po_copy')['value'].transform('min')) / (df_icicle_w_o_outlier.groupby('po_copy')['value'].transform('max') - df_icicle_w_o_outlier.groupby('po_copy')['value'].transform('min'))
df_icicle_w_o_outlier["year_copy"] = df_icicle_w_o_outlier["year"]
df_icicle_w_o_outlier = get_outlier(df_icicle_w_o_outlier,"year_copy", "po_copy","value_copy", 0.01)

df_icicle_w_o_outlier = df_icicle_w_o_outlier[df_icicle_w_o_outlier["outlier"]==0]

list_3 = ["lon","lat","posname","year", "value", 'n']

df_icicle_w_o_outlier = df_icicle_w_o_outlier[list_3]
print(df_icicle_w_o_outlier.shape)
df_icicle_w_o_outlier.to_csv(path_temp + 'sw'+str_sw_remark+'_outlier_1.csv',index=False)

In [ ]:
# weighted average of data based on sample size
# 根据样本数对数据进行加权平均
df_outlier = pd.read_csv(path_temp + 'sw'+str_sw_remark+'_outlier_1.csv')
print(df_outlier.shape)
list_1 = ["posname","lon","lat","year"]
list_2 = ["posname","lon","lat","year", "n","value"]
list_3 = ["posname","lon","lat","year", "value"]
df_icicle = df_outlier[list_2]

df_icicle_w = mf.get_weight_count(df_icicle,list_1,'mean') 

df_icicle_w_o = df_icicle_w[list_3]
print(df_icicle_w_o.shape)
df_icicle_w_o.to_csv(path_temp + 'sw'+str_sw_remark+'_weight_1.csv',index=False)

In [ ]:
# Data Gridding
# 数据网格化
df_weight = pd.read_csv(path_temp + 'sw'+str_sw_remark+'_weight_1.csv')
print(df_weight.shape)
int_grid = 1
def lat_lon_to_grid(lat, lon, grid_size=1):
    return round(lat / grid_size) * grid_size, round(lon / grid_size) * grid_size


df_weight['lat_grid'], df_weight['lon_grid'] = zip(*df_weight.apply(lambda row: lat_lon_to_grid(row['lat'], row['lon'], int_grid), axis=1))

df_grid = df_weight.groupby(['lat_grid', 'lon_grid', 'year', 'posname'])['value'].mean().reset_index()

print(df_grid.shape)
df_grid = df_grid.rename(columns={'posname':'id'})
posname_df = pd.read_csv(path_one_spdb + 'posname.csv')
posname_df = posname_df.drop(columns=["50%"])
df_grid = df_grid.merge(posname_df, how='left', on='id').drop('id', axis=1)

df_grid.to_csv(path_pre + 'sw'+str_sw_remark+'_grid_1.csv',index=False)